In [17]:
# Install required libraries
!pip install sentinelhub
!pip install rasterio

In [18]:
# Import necessary libraries
from google.colab import drive
import os

# Mount Google Drive to access files and save results
drive.mount('/content/drive')

# Create a folder named "Midecai" (no error if it already exists)
folder_path = "/content/drive/MyDrive/Midecai"
os.makedirs(folder_path, exist_ok=True)  # exist_ok=True prevents errors if the folder already exists

# Change the working directory to the created folder
os.chdir(folder_path)
print(f"Current directory: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current directory: /content/drive/MyDrive/Midecai


In [19]:
# Import required libraries
import numpy as np
import rasterio
import os
from datetime import datetime, timedelta
from rasterio.transform import from_origin
from sentinelhub import (
    BBox, CRS, bbox_to_dimensions, SentinelHubRequest,
    DataCollection, MosaickingOrder, MimeType, SentinelHubCatalog,
    filter_times, SHConfig
)
import threading
from concurrent.futures import ThreadPoolExecutor

In [20]:
# Evalscript to retrieve all 13 bands of Sentinel-2
evalscript_all_bands = """
//VERSION=3
function setup() {
    return {
        input: [{
            bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
            units: "DN"
        }],
        output: {
            bands: 13,
            sampleType: "INT16"
        }
    };
}

function evaluatePixel(sample) {
    return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05,
            sample.B06, sample.B07, sample.B08, sample.B8A, sample.B09,
            sample.B10, sample.B11, sample.B12];
}
"""

In [21]:
def save_as_geotiff(image_data, bbox, output_path):
    """Save the image with a transform ensuring NORTH is at the top."""
    min_lon, min_lat, max_lon, max_lat = bbox
    height, width, bands = image_data.shape

    # Calculate transform (origin at the NORTHWEST corner)
    res_lon = (max_lon - min_lon) / width
    res_lat = (max_lat - min_lat) / height  # Keep positive to ensure NORTH is at the top
    transform = from_origin(min_lon, max_lat, res_lon, res_lat)  # Do not use -res_lat

    # Set metadata
    meta = {
        'driver': 'GTiff',
        'dtype': image_data.dtype,
        'count': bands,
        'width': width,
        'height': height,
        'crs': 'EPSG:4326',
        'transform': transform
    }

    # Write the file
    with rasterio.open(output_path, 'w', **meta) as dst:
        for band in range(bands):
            dst.write(image_data[:, :, band], band + 1)

In [22]:
def split_bbox(bbox, n_rows=2, n_cols=2):
    """Split the BBox into smaller tiles, ordered from SOUTH to NORTH."""
    min_lon, min_lat, max_lon, max_lat = bbox
    lon_steps = np.linspace(min_lon, max_lon, n_cols + 1)
    lat_steps = np.linspace(min_lat, max_lat, n_rows + 1)  # From SOUTH to NORTH

    bboxes = []
    for row in range(n_rows):
        for col in range(n_cols):
            tile_bbox = (
                lon_steps[col],
                lat_steps[row],        # SOUTH boundary
                lon_steps[col + 1],
                lat_steps[row + 1]     # NORTH boundary
            )
            bboxes.append(BBox(tile_bbox, crs=CRS.WGS84))
    return bboxes

In [23]:
def download_tile(bbox, resolution, time_interval, config):
    """Download tile data and return a numpy array."""
    size = bbox_to_dimensions(bbox, resolution=resolution)

    request = SentinelHubRequest(
        evalscript=evalscript_all_bands,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=time_interval,
                mosaicking_order=MosaickingOrder.MOST_RECENT,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        config=config,
    )

    return request.get_data()[0]

In [24]:
def merge_tiles_in_memory(tile_data_list, n_rows, n_cols):
    """Merge tiles with NORTH at the top and SOUTH at the bottom."""
    # Sort tiles by row (NORTH to SOUTH) and column (WEST to EAST)
    sorted_tiles = sorted(tile_data_list, key=lambda x: (x[1], x[2]))

    # Merge rows from NORTH to SOUTH
    merged_rows = []
    for row in range(n_rows):
        row_tiles = [tile for tile in sorted_tiles if tile[1] == row]
        row_image = np.concatenate([tile[0] for tile in row_tiles], axis=1)
        merged_rows.append(row_image)

    # Reverse row order to ensure NORTH is at the top (raster images default to top-left origin)
    merged_image = np.concatenate(merged_rows[::-1], axis=0)  # Change here

    return merged_image

In [25]:
# Function to download a single tile (wrapped for threading)
def download_tile_threaded(tile_bbox, resolution, time_interval, config, row, col):
    image_data = download_tile(tile_bbox, resolution, time_interval, config)
    return image_data, row, col

In [ ]:
if __name__ == "__main__":
    # Authentication configuration
    config = SHConfig()
    CLIENT_ID = '2528275c-5c63-491d-b27e-91638e73a76d'  # Replace with your credentials
    CLIENT_SECRET = '03qMgDgm7QWoyr8nNzVqw9QpfWP7TGpF'  # Replace with your credentials

    if CLIENT_ID and CLIENT_SECRET:
        config.sh_client_id = CLIENT_ID
        config.sh_client_secret = CLIENT_SECRET

    # Region and time parameters
    DANANG_BBOX_COORDS = [108.136711, 15.925346, 108.351631, 16.183354]
    RESOLUTION = 10  # 10m/pixel resolution
    N_ROWS, N_COLS = 2, 2  # Split into 2x2 tiles
    TIME_DELTA = timedelta(hours=1)  # Time window for searching

    # Initialize catalog and search for images
    catalog = SentinelHubCatalog(config=config)
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=BBox(bbox=DANANG_BBOX_COORDS, crs=CRS.WGS84),
        time=("2024-01-01", "2024-03-31"),
        fields={"include": ["id", "properties.datetime"], "exclude": []},
    )
    unique_acquisitions = filter_times(search_iterator.get_timestamps(), time_difference=TIME_DELTA)

    # Process each acquisition timestamp
    for timestamp in unique_acquisitions:
        # Create output directory
        date = timestamp.date()
        week_num = (date.day - 1) // 7 + 1
        output_dir = os.path.join(
            f"Year_{date.year}",
            f"Month_{date.month:02d}",
            f"Week_{week_num}",
            f"Day_{date.day:02d}"
        )
        os.makedirs(output_dir, exist_ok=True)

        # Download and merge images
        tiles = split_bbox(BBox(DANANG_BBOX_COORDS, CRS.WGS84), N_ROWS, N_COLS)
        time_interval = (timestamp - TIME_DELTA, timestamp + TIME_DELTA)

        tile_data_list = []
        with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
            futures = []
            for idx, tile_bbox in enumerate(tiles):
                row = idx // N_COLS  # Row increases from SOUTH to NORTH
                col = idx % N_COLS   # Column increases from WEST to EAST
                futures.append(executor.submit(download_tile_threaded, tile_bbox, RESOLUTION, time_interval, config, row, col))

            for future in futures:
                tile_data_list.append(future.result())

        # Merge images
        merged_image = merge_tiles_in_memory(tile_data_list, N_ROWS, N_COLS)

        # Save the image
        output_path = os.path.join(output_dir, f"Danang_{date.isoformat()}.tif")
        save_as_geotiff(merged_image, DANANG_BBOX_COORDS, output_path)
        print(f"Processed image for {date.isoformat()}")

    print("Completed processing all images!")

Processed image for 2024-01-02
Processed image for 2024-01-07
Processed image for 2024-01-12
Processed image for 2024-01-17
Processed image for 2024-01-22
Processed image for 2024-01-27
Processed image for 2024-02-01
